## Data discovery: Some SQL Joins examples
> Download the dataset from [the official TLC Trip Record Data website](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

### This cell only shows how to document code
```python
# Load file
local_file = 'datasets/your-downloaded-from-TLC-taxis-file-here.parquet'

# Show data
spark.read.parquet(local_file).show()
```

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructField, StructType

### What is master(local N)?
The --master option specifies the master URL for a distributed cluster, or local to run locally with one thread, or local[N] to run locally with N threads.

<b>Source</b>: See Spark [docs here](spark.apache.org/docs/latest). See all [options here](https://spark.apache.org/docs/latest/submitting-applications.html#master-urls)

In [ ]:
# Create SparkSession
spark = SparkSession.builder\
             .master("local[1]")\
             .appName("spark-app-version-x")\
             .getOrCreate()

In [ ]:
# Read taxi data
local_file = 'datasets/parquet/'
df = spark.read.parquet(local_file)

In [ ]:
# DF is like a relation table in memory. Let's see the columns
df.printSchema()

In [ ]:
# Query sample, using Spark SQL
df.createOrReplaceTempView('tbl_raw_yellow_taxis')

In [ ]:
# SQL Statement
spark.sql('''
          select *
          from tbl_raw_yellow_taxis
          ''').show(n=5)

### Let's create a real "dimension" table, for our RateCodeID
1. Standard rate
2. JFK
3. Newark
4. Nassau or Westchester 
5. Negotiated fare
6. Group ride

In [ ]:
# Add Rate Code IDs
data = [("1", "Standard rate"), ("2", "JFK"), ("3", "Newark"),("4","Nassau or Westchester "),("5","Negotiated fare"), ("6","Group ride")]

In [ ]:
# Define schema, to ensure data types
schema = StructType([ \
    StructField("RatecodeID",StringType(),True), \
    StructField("RatecodeName",StringType(),True)
  ])

In [ ]:
# Create Dataframe for Rate Codes
df_rate_codes = spark.createDataFrame(data=data,schema=schema)

In [ ]:
# Show rates
df_rate_codes.show()

In [ ]:
# Inner join example:
df.join(df_rate_codes, df["RatecodeID"] == df_rate_codes["RatecodeID"], "inner").show(n=5)

In [ ]:
# Inner join example. Using only columns, to avoid duplicating columns from both tables
# - for example, see above how RatecodeID shows twice (one for each table joined)
df.join(df_rate_codes, df["RatecodeID"] == df_rate_codes["RatecodeID"], "left").drop(df_rate_codes["RatecodeID"]).show(n=5)

In [ ]:
# Left join example, showing those WITH NO rate code:
df.join(df_rate_codes, df["RatecodeID"] == df_rate_codes["rate_code_id"], "left").where("RatecodeID is NULL").show()

In [1]:
# SQL Statement
spark.sql('''
          select RatecodeID, count(1)
          from tbl_raw_yellow_taxis
          group by RatecodeID
          ''').show(n=5)

NameError: name 'spark' is not defined

In [ ]:
# If user doesn't want to see NULL values
df_na_rate_codes = df.na.fill(value=0,subset=["RatecodeID"])

In [ ]:
# Left join example, showing those WITH NO rate code:
df_na_rate_codes.join(df_rate_codes, df_na_rate_codes["RatecodeID"] == df_rate_codes["rate_code_id"], "left").where("RatecodeID is NULL").show()

In [ ]:
# Confirm counts
df_na_rate_codes.createOrReplaceTempView('tbl_na_rate_codes')

In [ ]:
# SQL Statement
spark.sql('''
          select RatecodeID, count(1)
          from tbl_na_rate_codes
          group by RatecodeID
          ''').show(n=5)

In [ ]:
# Stop the session
spark.stop()